In [3]:
from datasets import load_dataset, load_from_disk, concatenate_datasets
# from promptsource import templates
import json

CACHE_DIR = "/share/edc/home/antonis/datasets/huggingface"
import os
os.environ["HF_DATASETS_CACHE"] = CACHE_DIR

# # Get a list of all supported datasets
# datasets = templates.get_dataset_names()
# print(datasets)

/local/home/antonis/.conda/envs/incidental/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def print_example(example):
    for key in example.keys():
        print(f"{key}: {example[key]}")

In [ ]:
# ds = load_dataset("conceptofmind/dialog_submix_original", split="train")
# ds = load_dataset("conceptofmind/cot_submix_original", split="train")
# ds = load_dataset("conceptofmind/flan2021_submix_original", split="train")
ds = load_from_disk("/share/edc/home/antonis/datasets/huggingface/flan_v1/c4_mixed_NLI")

In [5]:
ds_task = load_from_disk('/share/edc/home/antonis/datasets/huggingface/flan_v1_task_ds_p_10')

In [6]:
ds_task

DatasetDict({
    NLI: Dataset({
        features: ['inputs', 'targets', 'task_source', 'task_name', 'template_type', 'text'],
        num_rows: 72402
    })
    QA: Dataset({
        features: ['inputs', 'targets', 'task_source', 'task_name', 'template_type', 'text'],
        num_rows: 1070
    })
    Summarization: Dataset({
        features: ['inputs', 'targets', 'task_source', 'task_name', 'template_type', 'text'],
        num_rows: 72254
    })
    Commonsense: Dataset({
        features: ['inputs', 'targets', 'task_source', 'task_name', 'template_type', 'text'],
        num_rows: 37262
    })
    Sentiment: Dataset({
        features: ['inputs', 'targets', 'task_source', 'task_name', 'template_type', 'text'],
        num_rows: 41361
    })
    Paraphrase: Dataset({
        features: ['inputs', 'targets', 'task_source', 'task_name', 'template_type', 'text'],
        num_rows: 25032
    })
    Reading Comp.: Dataset({
        features: ['inputs', 'targets', 'task_source', 'task_nam

In [ ]:
len(set(ds['task_name']))
set(ds['task_name'])

In [ ]:
required_tasks = [
    "anli",
    "rte",
    "cb",
    "snli",
    "mnli",
    "wnli",
    "qnli"
]

ds1 = ds.filter(lambda example: any(task in example['task_name'] for task in required_tasks))




In [ ]:
def concatenate_columns(dataset, column1, column2, new_column_name):
    def concat_example(example):
        example[new_column_name] = example[column1] + " " + example[column2]
        return example

    return dataset.map(concat_example)

# Usage example:
ds1 = concatenate_columns(ds1, 'inputs', 'targets', 'text')

In [ ]:
from transformers import GPT2Tokenizer
from tqdm import tqdm

def total_tokens(dataset, text_field):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    total_count = 0
    
    for example in tqdm(dataset, desc="Counting tokens"):
        text = example[text_field]
        tokens = tokenizer.tokenize(text)
        total_count += len(tokens)

    # Print the total count in scientific notation
    print(f"Total number of tokens: {total_count:.2e}")

    return total_count

# Usage example:
total_count = total_tokens(ds1, 'text')

In [ ]:
print(f"Total number of tokens: {total_count:.2e}")

In [ ]:
# ds['label']

# # get unique labels
# labels = set(ds['label'])
# print(labels)

In [ ]:
ds = load_from_disk("/share/edc/home/antonis/datasets/huggingface/merged_datasets/NLI/P_1_PQA_5_promptsource_False/dataset_1/dataset_train.arrow")

In [ ]:
labels = ds['train']['label']

# find -1 in labels
non_matching = []
for i in range(len(labels)):
    if labels[i] == -1:
        print(i)
        non_matching.append(i)
        print_example(ds['train'][i])

In [ ]:
# search revursively for .arrow file
import glob
import json

base_dir = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_unified_labels/P_1_PQA_5_promptsource_False"

arrow_files = glob.glob(base_dir + "/**/*.arrow", recursive=False)
print(arrow_files)

for file_ in arrow_files:
    print(file_)
    ds = load_from_disk(file_)
    print(ds)

json_files = glob.glob(base_dir + "/**/*.json", recursive=False)
print(json_files)


# def print_keys_values(data):
#     for key, value in data.items():
#         print(key)
#         if isinstance(value, dict):
#             print_keys_values(value)
#         else:
#             print(value)

for file_ in json_files:
    print(file_)
    with open(file_, 'r') as f:
        data = json.load(f)
    # print keys and values recursively
    # print_keys_values(data)
    print(data)



In [ ]:
arrow_files

In [ ]:
# ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/dataset_1/dataset_train.arrow"
# ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment/dataset_1/dataset_validation.arrow"
# ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment/P_QA_5/dataset_1/dataset_validation.arrow"
# dataset_c4 = load_dataset("c4", "en", split="train[:20%]", cache_dir=CACHE_DIR)

ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_1/dataset_train.arrow"
dataset = load_from_disk(ds_pth)

# ds_path2 = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_0/dataset_train.arrow"
# dataset2 = load_from_disk(ds_path2)

ds_pth_validation = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_1/dataset_validation.arrow"
dataset_validation = load_from_disk(ds_pth_validation)

In [ ]:
len(ds)

In [ ]:
ds_imdb = load_dataset("imdb", split="test", cache_dir=CACHE_DIR)
ds_sst = load_dataset("sst", split="test", cache_dir=CACHE_DIR)

print(f"IMDB len: {len(ds_imdb)}")
print(f"SST len: {len(ds_sst)}")

In [ ]:
# first protion is IMDB, second is SST
ds_pth_val = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_1/dataset_validation.arrow"
dataset_val = load_from_disk(ds_pth_val)
ds_pth2_val = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_0/dataset_validation.arrow"
dataset2_val = load_from_disk(ds_pth2_val)

In [ ]:
import json

ds_path = "/share/edc/home/antonis/datasets/huggingface/C4/limit_total_tokens_1240000000/train"
ds = load_from_disk(ds_path)

In [ ]:
print(f"train1: {len(dataset)}")
print(f"train2: {len(dataset2)}")
print(f"val1: {len(dataset_val)}")
print(f"val2: {len(dataset2_val)}")

In [ ]:
dataset_val.select(range(len(dataset2_val)))

In [ ]:
# dataset_val[-len(dataset2_val):]

dataset_val1 = dataset_val.select(range(len(ds_imdb)))
dataset_val2 = dataset_val.select(range(len(ds_imdb), len(dataset_val)))

assert len(dataset_val1) + len(dataset_val2) == len(dataset_val), f"len mismatch: {len(dataset_val1)} + {len(dataset_val2)} != {len(dataset_val)}"

def add_new_column1(example, keyword, column_name="dataset"):
    example[column_name] = keyword
    return example

dataset_val1 = dataset_val1.map(lambda x: add_new_column1(x, "imdb"))
dataset_val2 = dataset_val2.map(lambda x: add_new_column1(x, "sst"))

dataset_val = concatenate_datasets([dataset_val1, dataset_val2])

# save the new dataset
dataset_val.save_to_disk(os.path.join(os.path.dirname(ds_pth_val), "dataset_validation_wname.arrow"))

In [ ]:
print(os.path.join(os.path.dirname(ds_pth_val), "dataset_validation_wname.arrow"))

In [ ]:
import os
import json
import logging
from datasets import load_dataset, Dataset
from promptsource.templates import DatasetTemplates, TemplateCollection
# set logging level to INFO
from src._promptsource import get_T0MixtureDatasets 
logger = logging.getLogger(__name__)
logger.setLevel(20)

In [ ]:
TOMixture = [
    ("sst","default"), # Senitment Classification")
    ("imdb",None),
]

In [ ]:
# datasets = get_T0MixtureDatasets("test", max_samples=1000, return_as_dict=True)
# ds_sst = load_dataset("sst", "default", split="test", cache_dir=CACHE_DIR)
ds_imdb = load_dataset("imdb", split="test", cache_dir=CACHE_DIR)
# ds_yelp_review = load_dataset("yelp_review_full", split="train", cache_dir=CACHE_DIR)
ds_sentiment140 = load_dataset("sentiment140", split="test", cache_dir=CACHE_DIR)

print(f"IMDB len: {len(ds_imdb)}")
# print(f"Yelp len: {len(ds_yelp_review)}")
print(f"Sentiment140 len: {len(ds_sentiment140)}")

In [ ]:
name, subset = TOMixture[0]
example = ds_sst.select(range(1))
# DatasetTemplates(name, subset).templates.get_get_answer_choices_list(example)

In [ ]:
print(len(ds_sst))
print(len(ds_imdb))
# print(len(ds_yelp_review))
# print(len(ds_sentiment140))

import random

ds_name = datasets['imdb']

def apply_template(example, template, dataset_name):
    if isinstance(template, list):
        template = random.choice(template)
    try:
        answer_choices = template.get_answer_choices_list(example)
    except:
        answer_choices = None
    # example['label'] = float(example['label'])
    return {"prompt": template.apply(example), "answer_choices": answer_choices, "dataset": dataset_name}

# # You can apply the template to the dataset
# prompted_dataset = ds_name.map(lambda example: apply_template(example, ds_name.templates))

def create_prompted_dataset(datasets):
    prompted_datasets = {}
    for name, dataset in datasets.items():
        prompted_dataset = dataset.map(lambda example: apply_template(example, dataset.templates, name))
        # columns to keep
        columns_to_keep = ["prompt", "answer_choices", "dataset"]
        prompted_dataset = prompted_dataset.remove_columns([col for col in prompted_dataset.column_names if col not in columns_to_keep])
        prompted_datasets[name] = prompted_dataset

    prompted_datasets = concatenate_datasets([dataset for name, dataset in prompted_datasets.items()])
    return prompted_datasets

prompted_datasets = create_prompted_dataset(datasets)

In [ ]:
# save prompted dataset

# prompted_datasets.save_to_disk("/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_0/dataset_test_with_answers.arrow")

In [ ]:
for name, dataset in datasets.items():
    print(name, dataset)
    print(dataset.templates)